In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [21]:
df=pd.read_csv('Churn_Modelling.csv')
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [22]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [23]:
label_encoder_gender=LabelEncoder()
df["Gender"]=label_encoder_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [24]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(df[['Geography']])
onehot_encoder_geo.get_feature_names_out(["Geography"])
df_geography=pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
df=pd.concat([df.drop('Geography',axis=1),df_geography],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [25]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)


In [27]:
## devide dataset into dependent and indepndent
X=df.drop(['Exited'],axis=1)
Y=df['Exited']
#spli data in train and test
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [29]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [30]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [31]:
## build our ANN model
#1 Squential model- insquential use some layer for which we will use dense and with dense will create hidden layer
## we are basically initializing our input along with dense layer
model=Sequential([
## we wil tell dense to implent 64 neuron and activate it(forthe first layer we need to write input shape)
Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL2 connected with input layer
Dense(64,activation='relu'),## HL2
Dense(1,activation='sigmoid') ## output layer
])

In [32]:
## forward and backward propagation we have to complie the model
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [33]:
## setup tensoboard so first create directrory
log_dir="logs/fit" +datetime.datetime.now().strftime("%Y%d%m-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [35]:
## setup early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights='True')

In [36]:
#Train the model
history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
                  callbacks=[tensorflow_callback,early_stopping_callback]
                  )

Epoch 1/100
250/250 [==============================] - 5s 5ms/step - loss: 0.4570 - accuracy: 0.8012 - val_loss: 0.3936 - val_accuracy: 0.8355
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3767 - accuracy: 0.8479 - val_loss: 0.3546 - val_accuracy: 0.8510
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3514 - accuracy: 0.8561 - val_loss: 0.3451 - val_accuracy: 0.8605
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3430 - accuracy: 0.8595 - val_loss: 0.3446 - val_accuracy: 0.8575
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3389 - accuracy: 0.8612 - val_loss: 0.3462 - val_accuracy: 0.8625
Epoch 6/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3356 - accuracy: 0.8611 - val_loss: 0.3429 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3335 - accuracy: 0.8633 - val_loss: 0.3422 - val_accuracy: 0.8585

In [37]:
model.save('model.h5')

c:\Users\VISHES\ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
## TensorBoard extension
%reload_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [39]:
%tensorboard --logdir logs/fit20252904-071200

In [ ]:
## load pickel file